#Imports

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import itertools
import time
from google.colab import drive

In [ ]:
drive.mount('/content/drive')
%cd /content/drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


#Functions

In [ ]:
def href(soup):
  # get all href links from one page
  href=[]
  for i in soup.find_all("a",class_="s-link",href=True):
    if i['href'] != 'javascript:void(0)' :
      href.append(i['href'])
  return href

In [ ]:
def clean_empty_hrefs(hrefs):
  # remove all empty lists
  list_hrefs=[]
  for i in hrefs:
      if i!=[]:
          list_hrefs.append(i)
  # merge all elemenets in one list
  herfs_list=[]
  for i in list_hrefs:
    for j in i:
        herfs_list.append(j)
  return herfs_list

In [ ]:
def add_prefix(herfs_list,prefix):
  new_href=[]
  for h in herfs_list:
    m=prefix+h
    new_href.append(m)
  return new_href

In [ ]:
def single_page_scraper(url):
  req=requests.get(url=url)
  time.sleep(1.5)
  soup=BeautifulSoup(req.text,"html.parser")
  return soup

In [ ]:
def single_page_question_answer(url):
  #Get the Page Soup
  page=single_page_scraper(url)
  #Get The Question Title
  question = page.find_all("a",class_="question-hyperlink")[0].get_text()
  #Get All the Answer Divs
  AnswerDivs=page.find_all("div",class_="s-prose js-post-body")[1:]
  answer=["\n".join([i.get_text() for i  in AnswerDiv.find_all("p")]) for AnswerDiv in AnswerDivs]
  return question,answer

In [ ]:
quesitons_answers_List=[]
hrefs=[]
soups=[]
def questions_answers(start_page,end_page,prefix):

  #Obtain All Soups For The Question Pages
  for page in range(start_page,end_page):
    print('We Are Now In Page:{}'.format(page))
    time.sleep(1.5)
    req=requests.get(url='https://'+prefix+'/questions?tab=votes&page='+str(page))
    soup=BeautifulSoup(req.text,"html.parser")
    soups.append(soup)

  print("Soups Are Ready!")

  #Obtain All Hrefs From The Question Pages
  for soup in soups:
      hrefs.append(href(soup))
      herfs_list=clean_empty_hrefs(hrefs)
  new_hrefs_list=add_prefix(herfs_list,prefix)
  print("All Hrefs Are Ready!")
  print("We Will Now Collect The Data, From {} Questions".format(len(new_hrefs_list)))

  #Obtain The Questions And Answers
  for i in len(new_hrefs_list):
    try:
      q,a=single_page_question_answer(new_hrefs_list[i])
      for j in range(len(a)):
        quesitons_answers_List.append([q,a[j]])
        if len(quesitons_answers_List)%500==0:
          df = pd.DataFrame(quesitons_answers_List, columns = ["Question",'Answer'])
          df.to_csv('NLP_Data.csv',mode = 'w')
    except:
      continue
    print("After {} Questions, We Have Collected {} Question/Answers Pairs".format(len(quesitons_answers_List,i)))
  df = pd.DataFrame(quesitons_answers_List, columns = ["Question",'Answer'])
  df.to_csv('NLP_Data.csv',mode = 'w')
  print("Quesitons&Answers Are Ready!")
  return quesitons_answers_List

#Running

In [ ]:
prefix='https://lifehacks.stackexchange.com'
quesitons_answers_List=questions_answers(0,199,prefix)

We Have Collected 5836 Question/Answers Pair
We Have Collected 5839 Question/Answers Pair
We Have Collected 5842 Question/Answers Pair
We Have Collected 5845 Question/Answers Pair
We Have Collected 5847 Question/Answers Pair
We Have Collected 5849 Question/Answers Pair
We Have Collected 5851 Question/Answers Pair
We Have Collected 5854 Question/Answers Pair
We Have Collected 5858 Question/Answers Pair
We Have Collected 5861 Question/Answers Pair
We Have Collected 5862 Question/Answers Pair
We Have Collected 5865 Question/Answers Pair
We Have Collected 5866 Question/Answers Pair
We Have Collected 5867 Question/Answers Pair
We Have Collected 5874 Question/Answers Pair
We Have Collected 5880 Question/Answers Pair
We Have Collected 5884 Question/Answers Pair
We Have Collected 5888 Question/Answers Pair
We Have Collected 5894 Question/Answers Pair
We Have Collected 5897 Question/Answers Pair
We Have Collected 5905 Question/Answers Pair
We Have Collected 5915 Question/Answers Pair
We Have Co